## Visual 5
To see this visual, first change the data_path variable to the correct path. Then, run all cells, and finally the visual will open in the browser, where you can see it and interact with it. 

In [1]:
import pandas as pd
import numpy as np
import warnings

In [2]:
# DATA LOADING CELL
data_path = "..\\Crimedata.csv"
data = pd.read_csv(data_path) 

c:\Users\Teodora\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (15,40,41,42,43,44,45,46,52,53,54,71,77,78,79,80,82,83,84,85,101,102,103,104,105,130,131,137,153,154,155,156,157,163,164,165,181,182,183,184,191,207,208,209,213,217,233,234,235,236,237,261,262,263,285,286,287,291,307,308,309,310,311,336,337,338,339,361,362,363,366,367,383,384,385,386,387,419,480,547,555,561,562,563,732,734,736,740,752,754,756,760,800,865,866,867,868,869,870,891,917,965,1006,1007,1085,1086,1087,1148,1167,1218,1219,1243,1245,1246,1248,1251,1252,1257,1260,1269,1285,1326,1328,1338,1340,1350,1352,1354,1368,1397,2084,2085,2086,2091) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
col_dict = {'V1267': 'Alcohol',
'V1266' : "Alcohol_12months",
 'V1327': 'Marijuana',
 'V1315' : 'Marijuana_30days',
 'V1339' : 'Marijuana_12months',
 'V1330': 'Heroin',
 'V1318' : 'Heroin_30days',
 'V1342' : 'Heroin_12months',
 'V1328': 'Cocaine',
 'V1316': 'Cocaine_30days',
 'V1340': 'Cocaine_12months',
 'RV0052' : 'Highest_Year_Education_Before_Prison',
 'RV0001': 'Age',
 'RV0003': 'Race',
 'RV0005': 'Sex',
 'V0772': 'State'} # State where living at time of arrest

In [4]:
data.rename(columns=col_dict,inplace=True)

In [5]:
data.shape

(24848, 2105)

In [ ]:
cols_to_clean = ["Alcohol","Marijuana","Heroin","Cocaine"]
df = data[["Alcohol","Marijuana","Heroin","Cocaine","Age","Sex","Race","State", 'Highest_Year_Education_Before_Prison']]
for col in cols_to_clean:
    df[col] = df[col].str.extract(r'([\-0-9]+)').fillna(0).astype(int)
df['Sex'] = df['Sex'].str.extract(r'([A-Za-z]+)')
df['Race'] = df['Race'].str.slice(7)
df.to_csv("./extracted_data_vis5.csv",index=False)

In [2]:
df = pd.read_csv("extracted_data_vis5.csv")

In [3]:
df.head()

,Alcohol,Marijuana,Heroin,Cocaine,Age,Sex,Race,State,Highest_Year_Education_Before_Prison
0,2,0,0,0,51,Male,White (NH),GA,12
1,2,1,0,0,29,Male,White (NH),SC,8
2,2,2,0,2,43,Female,White (NH),IN,15
3,1,2,2,2,45,Male,White (NH),CA,10
4,2,0,0,0,31,Female,Black (NH),OH,13


In [3]:
# remove missing values in years of education
na_values = [99, 98]
df = df[~df.Highest_Year_Education_Before_Prison.isin(na_values)]

In [4]:
drug_data = df.groupby('Highest_Year_Education_Before_Prison')["Alcohol", 'Marijuana', 'Cocaine', 'Heroin'].apply(lambda r: r[r==1].count())
drug_data["total_prisoners"] = df.groupby('Highest_Year_Education_Before_Prison')['Alcohol'].count().values
drug_data.reset_index(inplace=True)
drug_data.head()

C:\Users\Dell\AppData\Local\Temp\ipykernel_16740\3565264193.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  drug_data = df.groupby('Highest_Year_Education_Before_Prison')["Alcohol", 'Marijuana', 'Cocaine', 'Heroin'].apply(lambda r: r[r==1].count())


,Highest_Year_Education_Before_Prison,Alcohol,Marijuana,Cocaine,Heroin,total_prisoners
0,0,20,10,6,0,81
1,1,21,17,7,4,98
2,2,26,13,14,4,92
3,3,30,7,6,4,100
4,4,24,23,11,7,95


In [5]:
import matplotlib
print(matplotlib.rcsetup.interactive_bk)

['GTK3Agg', 'GTK3Cairo', 'GTK4Agg', 'GTK4Cairo', 'MacOSX', 'nbAgg', 'QtAgg', 'QtCairo', 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo']


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.switch_backend('WebAgg')

def main():
    subplots = ZoomingSubplots(4, 1)
    colors = ['#780000', '#c1121f', '#003049', '#669bbc']
    labels = ['Alcohol', 'Marijuana', 'Cocaine', 'Heroin']
    present_data = [drug_data['Alcohol'], drug_data['Marijuana'], drug_data['Cocaine'], drug_data['Heroin']]
    i = -1
    for ax, color in zip(subplots.axes.flat, colors):
        i += 1
        x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
        data = present_data[i]
        ax.set_ylabel(labels[i])
        ax.scatter(x, data, color=color)
        if(i==3): 
            ax.get_xaxis().set_visible(True)    
        else:
            ax.get_xaxis().set_visible(False)
        
    plt.ion()
    subplots.fig.supxlabel('Years of Education before Prison')
    subplots.fig.suptitle('Click on an axes to make it fill the figure.\n'
                 'Click again to restore it to its original position')
    
    plt.show()
    #plt.subplots_adjust()

class ZoomingSubplots(object):
    def __init__(self, *args, **kwargs):
        """All parameters passed on to 'subplots`."""
        self.fig, self.axes = plt.subplots(*args, **kwargs)
        self._zoomed = False
        self.fig.canvas.mpl_connect('button_press_event', self.on_click)

    def zoom(self, selected_ax):
        for ax in self.axes.flat:
            ax.set_visible(False)
        self._original_size = selected_ax.get_position()
        selected_ax.set_position([0.125, 0.1, 0.775, 0.8])
        selected_ax.set_visible(True)
        self._zoomed = True

    def unzoom(self, selected_ax):
        selected_ax.set_position(self._original_size)
        for ax in self.axes.flat:
            ax.set_visible(True)
        self._zoomed = False

    def on_click(self, event):
        if event.inaxes is None:
            return
        if self._zoomed:
            self.unzoom(event.inaxes)
        else:
            self.zoom(event.inaxes)
        self.fig.canvas.draw()

if __name__ == '__main__':
    main()

In [15]:
class ZoomingSubplots(object):
    def __init__(self, *args, **kwargs):
        """All parameters passed on to 'subplots`."""
        self.fig, self.axes = plt.subplots(*args, **kwargs)
        self._zoomed = False
        self.fig.canvas.mpl_connect('button_press_event', self.on_click)

    def zoom(self, selected_ax):
        for ax in self.axes.flat:
            ax.set_visible(False)
        self._original_size = selected_ax.get_position()
        selected_ax.set_position([0.125, 0.1, 0.775, 0.8])
        selected_ax.set_visible(True)
        self._zoomed = True

    def unzoom(self, selected_ax):
        selected_ax.set_position(self._original_size)
        for ax in self.axes.flat:
            ax.set_visible(True)
        self._zoomed = False

    def on_click(self, event):
        if event.inaxes is None:
            return
        if self._zoomed:
            self.unzoom(event.inaxes)
        else:
            self.zoom(event.inaxes)
        self.fig.canvas.draw()

In [8]:
def myplot():
    subplots = ZoomingSubplots(4, 1)
    colors = ['#780000', '#c1121f', '#003049', '#669bbc']
    labels = ['Alcohol', 'Marijuana', 'Cocaine', 'Heroin']
    present_data = [drug_data['Alcohol'], drug_data['Marijuana'], drug_data['Cocaine'], drug_data['Heroin']]
    i = -1
    for ax, color in zip(subplots.axes.flat, colors):
        i += 1
        x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
        data = present_data[i]
        ax.set_ylabel(labels[i])
        ax.scatter(x, data, color=color)
        if(i==3): 
            ax.get_xaxis().set_visible(True)    
        else:
            ax.get_xaxis().set_visible(False)
        
    plt.ion()
    subplots.fig.supxlabel('Years of Education before Prison')
    subplots.fig.suptitle('Click on an axes to make it fill the figure')
    
    plt.show()

In [12]:
import numpy as np
import matplotlib.pyplot as plt
plt.switch_backend('WebAgg')

In [19]:
plt.show()